In [4]:

import sys
sys.path.insert(0, '/Users/james/Deep-Reinforcement-Learning-for-Hedging/')
import numpy as np
import torch
import matplotlib.pyplot as plt
from financial_models.asset_price_models import GBM
from financial_models.option_price_models import BSM
from torch import nn
from hedging_env_gymnasium import HedgingEnv
from baseline_agent import DeltaNeutralAgent


seed = 345
np.random.seed(seed)
torch.manual_seed(seed)

mu = 0.05
dt = 1/128
T = 1
num_steps = T/dt
s_0 = 1
strike_price = s_0
sigma = 0.15
r = 0.01

apm = GBM(mu=mu, dt=dt, s_0=s_0, sigma=sigma)
opm = BSM(strike_price=strike_price, risk_free_interest_rate=r, volatility=sigma, T=T, dt=dt)
env = HedgingEnv(asset_price_model=apm, dt=dt, T=T, num_steps=num_steps, trading_cost_para=1,
                 L=1, strike_price=strike_price, int_holdings=False, initial_holding=0, mode="PL",
                 option_price_model=opm)


In [8]:


# Initialize agent
agent = DeltaNeutralAgent(max_holding=env.max_holding)

# Run simulations
num_simulations = 1000
reward_array = np.zeros(num_simulations)
for i in range(num_simulations):
    episode_reward = 0
    state,_ = env.reset()
    done = False
    while not done:
        action = agent.act(state)
        state, reward, done, _ ,_= env.step(action)
        episode_reward += reward
    reward_array[i] = episode_reward

reward_array.mean()

3.1642730139167643